In [2]:
import geodesic
import yaml
from geodesic.tesseract import Step, StepInput, StepOutput, Container
import numpy as np
import ee

In [10]:
ee_creds = ee.ServiceAccountCredentials('earth-engine@double-catfish-291717.iam.gserviceaccount.com', '/home/rob/.config/gcloud/gee.json')
ee.Initialize(ee_creds)

In [3]:
proj = geodesic.create_project(
    name="har-reg",
    alias="Harmonic Regression Demo",
    description="A demo of harmonic regression models in Entanglement",
    keywords=["demo", "harmonic regression", "entanglement"],
    )
geodesic.set_active_project(proj)

{'name': 'har-reg',
 'alias': 'Harmonic Regression Demo',
 'description': 'A demo of harmonic regression models in Entanglement',
 'keywords': 'demo, harmonic regression, entanglement',
 'uid': '4941c7671b52e1af02090fcae36a301c7129d974',
 'owner': 'auth0|60a83a706acb9a00718c598a'}

In [3]:
geodesic.get_objects()

[modelset:remote-sensing:earth-observation:multispectral:harmonic-regression-model]

In [4]:
with open('harreg_modelset.yaml') as f:
    modelsets = yaml.safe_load(f)

In [5]:
modelsets['modelset']['tesseract_config']

{'name': 'harmonic-regression',
 'type': 'model',
 'inputs': [{'asset_name': 'input-imagery',
   'spatial_chunk_shape': [512, 512],
   'overlap': 0,
   'bands': [0, 1, 2, 3, 4, 5],
   'type': 'tensor'}],
 'outputs': [{'asset_name': 'brightness-parameters',
   'chunk_shape': [1, 10, 512, 512],
   'type': 'tensor',
   'pixel_dtype': '<f8',
   'fill_value': 'nan'},
  {'asset_name': 'greenness-parameters',
   'chunk_shape': [1, 10, 512, 512],
   'type': 'tensor',
   'pixel_dtype': '<f8',
   'fill_value': 'nan'},
  {'asset_name': 'wetness-parameters',
   'chunk_shape': [1, 10, 512, 512],
   'type': 'tensor',
   'pixel_dtype': '<f8',
   'fill_value': 'nan'}]}

In [6]:
cli = geodesic.service.ServiceClient('entanglement', '1')

In [7]:
modelsets['modelset']['project'] = proj.uid

In [9]:
resp = cli.post('modelsets',project=proj.uid, **modelsets)
resp.json()

{'uids': ['0x42aeb4']}

In [19]:
objs = geodesic.get_objects()
objs

[modelset:remote-sensing:earth-observation:multispectral:harmonic-regression-model,
 dataset:earth-observation:satellite:electro-optical:landsat-8]

In [20]:
model = objs[0]

In [5]:
# for obj in objs:
    # obj.delete(hard=True, show_prompt=False)

In [15]:
creds = geodesic.account.Credential.from_gcp_service_account(name="gee-creds")

In [17]:
creds.create()

In [11]:
landsat = geodesic.Dataset.from_google_earth_engine(
    name='landsat-8',
    asset='LANDSAT/LC08/C02/T1_L2',
    credential='gee-creds',
    alias='Landsat 8 Surface Reflectance',
    domain='earth-observation',
    category='satellite',
    type='electro-optical',
)
# landsat.save()

In [18]:
landsat.save()

ObjectWidget(object_value=[{'alias': 'Landsat 8 Surface Reflectance', 'name': 'landsat-8', 'project': '4941c76…

In [26]:
model_input = geodesic.entanglement.Predicate(
    name="model-input",
)

In [27]:
geodesic.entanglement.add_predicates('Input', [model_input])

In [38]:
conn = landsat.connect_with(predicate='model-input', object=model)

Going to just grab the dataset from the prod cluster for the har-reg tesseract output and put it into the test cluster. Dont want to have to rerun the tesseract job if I dont have to.

In [2]:
proj = geodesic.set_active_project('har-reg')

In [3]:
proj

{'name': 'har-reg',
 'alias': 'harmonic regression demo',
 'description': 'demo of harmonic regression',
 'owner': 'auth0|60a83a706acb9a00718c598a',
 'keywords': 'TCT, harmonic regression, demo',
 'uid': '4941c7671b52e1af02090fcae36a301c7129d974'}

In [5]:
objs = geodesic.get_objects()
objs

[dataset:earth-bservation:satellite:electro-optical:landsat-8,
 dataset:*:*:*:har-reg-demo,
 dataset:*:*:*:har-reg-demo-full,
 dataset:*:*:*:har-reg-demo-large-area]

In [6]:
ds = objs[2]

In [5]:
import json

In [11]:
with open('har-reg-demo-full.json', 'w') as f:
    json.dump(ds, f, indent=2)

Switched back to the test cluster so Im just going to add this dataset.

In [12]:
all = geodesic.get_objects()
all

[modelset:remote-sensing:earth-observation:multispectral:harmonic-regression-model,
 dataset:earth-observation:satellite:electro-optical:landsat-8,
 dataset:*:*:*:har-reg-demo-full]

In [13]:
model_node = all[0]

In [8]:
with open('har-reg-demo-full.json') as f:
    dsj = json.load(f)

In [9]:
ds = geodesic.Dataset(**dsj)

In [10]:
ds.save()

ObjectWidget(object_value=[{'alias': 'Harmonic Regression Demo', 'name': 'har-reg-demo-full', 'project': '4941…

In [14]:
ds.connect_with(predicate='derived-from', object=model_node)

dataset:*:*:*:har-reg-demo-full --derived-from--> modelset:remote-sensing:earth-observation:multispectral:harmonic-regression-model

In [15]:
usfs = geodesic.Object(
    name="us-forest-service",
    alias="US Forest Service",
    description="A US government agency responsible for managing the nation's forests and grasslands.",
    keywords=["forest", "management", "government"],
    object_class="Entity",
    domain="us-governement",
    category="government-agency",
    type="forest-management",
    item={
        'href': 'https://www.fs.usda.gov/',
    },
    project=proj.uid
)
usfs.save()

entity:us-governement:government-agency:forest-management:us-forest-service

In [16]:
forest_health = geodesic.Object(
    name="forest-health",
    alias="Forest Health",
    description="The condition of a forest, including the presence of pests, diseases, and other stressors.",
    keywords=["forest", "health", "condition"],
    object_class="Concept",
    domain="ecology",
    category="forest-ecology",
    type="forest-condition",
    project=proj.uid
)
forest_health.save()

concept:ecology:forest-ecology:forest-condition:forest-health

In [17]:
landcover = geodesic.Object(
    name="landcover",
    alias="Land Cover",
    description="The physical and biological cover of the earth's surface, including vegetation, water, and other features.",
    keywords=["land", "cover", "vegetation"],
    object_class="Concept",
    domain="earth-observation",
    category="land-cover",
    type="land-cover",
    project=proj.uid
)
landcover.save()

concept:earth-observation:land-cover:land-cover:landcover

In [18]:
usfs.connect_with(predicate='responsible-for', object=forest_health)
usfs.connect_with(predicate='related-to', object=landcover)
model_node.connect_with(predicate='can-observe', object=landcover)
model_node.connect_with(predicate='can-observe', object=forest_health)

modelset:remote-sensing:earth-observation:multispectral:harmonic-regression-model --can-observe--> concept:ecology:forest-ecology:forest-condition:forest-health

In [19]:
eo_imagery = geodesic.Object(
    name="eo-imagery",
    alias="Earth Observation Imagery",
    description="Imagery of the earth's surface captured by satellites and other remote sensing platforms.",
    keywords=["earth", "observation", "imagery"],
    object_class="Concept",
    domain="earth-observation",
    category="imagery",
    type="earth-observation-imagery",
    project=proj.uid
)
eo_imagery.save()

concept:earth-observation:imagery:earth-observation-imagery:eo-imagery

In [26]:
red = geodesic.Object(
    name="red",
    alias="Red Band",
    description="The red band of the electromagnetic spectrum, typically used for vegetation analysis or combined with other bands to create visible light imagery products. Usually centered around 650 nm wavelength, but can range from 630-675 nm.",
    keywords=["red", "band", "vegetation"],
    object_class="Observable",
    domain="earth-observation",
    category="spectral-band",
    type="red-band",
    project=proj.uid
)
red.save()

observable:earth-observation:spectral-band:red-band:red

In [32]:
blue= geodesic.Object(
    name="blue",
    alias="Blue Band",
    description="The blue band of the electromagnetic spectrum, typically used for water quality analysis or combined with other bands to create visible light imagery products. Usually centered around 450 nm wavelength, but can range from 430-515 nm.",
    keywords=["blue", "band", "water"],
    object_class="Observable",
    domain="earth-observation",
    category="spectral-band",
    type="blue-band",
    project=proj.uid
)
blue.save()

observable:earth-observation:spectral-band:blue-band:blue

In [33]:
green = geodesic.Object(
    name="green",
    alias="Green Band",
    description="The green band of the electromagnetic spectrum, typically used for vegetation analysis or combined with other bands to create visible light imagery products. Usually centered around 550 nm wavelength, but can range from 500-590 nm.",
    keywords=["green", "band", "vegetation"],
    object_class="Observable",
    domain="earth-observation",
    category="spectral-band",
    type="green-band",
    project=proj.uid
)
green.save()

observable:earth-observation:spectral-band:green-band:green

In [34]:
nir = geodesic.Object(
    name="nir",
    alias="Near Infrared Band",
    description="The near infrared band of the electromagnetic spectrum, typically used for vegetation analysis or combined with other bands to create visible light imagery products. Usually centered around 850 nm wavelength, but can range from 800-900 nm.",
    keywords=["nir", "band", "vegetation"],
    object_class="Observable",
    domain="earth-observation",
    category="spectral-band",
    type="nir-band",
    project=proj.uid
)
nir.save()

observable:earth-observation:spectral-band:nir-band:nir

In [35]:
swir1 = geodesic.Object(
    name="swir1",
    alias="Shortwave Infrared Band 1",
    description="The shortwave infrared band of the electromagnetic spectrum, typically used for vegetation analysis or combined with other bands to create visible light imagery products. Usually centered around 1650 nm wavelength, but can range from 1500-1750 nm.",
    keywords=["swir1", "band", "vegetation"],
    object_class="Observable",
    domain="earth-observation",
    category="spectral-band",
    type="swir1-band",
    project=proj.uid
)
swir1.save()

observable:earth-observation:spectral-band:swir1-band:swir1

In [36]:
swir2 = geodesic.Object(
    name="swir2",
    alias="Shortwave Infrared Band 2",
    description="The shortwave infrared band of the electromagnetic spectrum, typically used for vegetation analysis or combined with other bands to create visible light imagery products. Usually centered around 2200 nm wavelength, but can range from 2100-2300 nm.",
    keywords=["swir2", "band", "vegetation"],
    object_class="Observable",
    domain="earth-observation",
    category="spectral-band",
    type="swir2-band",
    project=proj.uid
)
swir2.save()

observable:earth-observation:spectral-band:swir2-band:swir2

In [62]:
b = geodesic.get_objects(search="landsat")
b

[dataset:earth-observation:satellite:electro-optical:landsat-8,
 dataset:*:*:*:har-reg-demo-full]

In [63]:
b[0].connect_with(predicate='can-observe', object=red)
b[0].connect_with(predicate='can-observe', object=blue)
b[0].connect_with(predicate='can-observe', object=green)
b[0].connect_with(predicate='can-observe', object=nir)
b[0].connect_with(predicate='can-observe', object=swir1)
b[0].connect_with(predicate='can-observe', object=swir2)

dataset:earth-observation:satellite:electro-optical:landsat-8 --can-observe--> observable:earth-observation:spectral-band:swir2-band:swir2

In [65]:
eo = geodesic.get_objects(search="eo-imagery")
eo

[concept:earth-observation:imagery:earth-observation-imagery:eo-imagery]

In [67]:
landsat = geodesic.get_objects(search="landsat")[0]
print(landsat)

dataset:earth-observation:satellite:electro-optical:landsat-8


In [68]:
eo[0].connect_with(predicate='related-to', object=landsat)

concept:earth-observation:imagery:earth-observation-imagery:eo-imagery --related-to--> dataset:earth-observation:satellite:electro-optical:landsat-8

In [69]:
paper = geodesic.Object(
    name="har-reg-usfs-paper",
    alias="Harmonic regression of Landsat time series for modeling attributes from national forest inventory data",
    description="Imagery from the Landsat Program has been used frequently as a source of auxiliary data for modeling land cover, as well as a variety of attributes associated with tree cover. With ready access to all scenes in the archive since 2008 due to the USGS Landsat Data Policy, new approaches to deriving such auxiliary data from dense Landsat time series are required. Several methods have previously been developed for use with finer temporal resolution imagery (e.g. AVHRR and MODIS), including image compositing and harmonic regression using Fourier series. The manuscript presents a study, using Minnesota, USA during the years 2009–2013 as the study area and timeframe. The study examined the relative predictive power of land cover models, in particular those related to tree cover, using predictor variables based solely on composite imagery versus those using estimated harmonic regression coefficients. The study used two common non-parametric modeling approaches (i.e. k-nearest neighbors and random forests) for fitting classification and regression models of multiple attributes measured on USFS Forest Inventory and Analysis plots using all available Landsat imagery for the study area and timeframe. The estimated Fourier coefficients developed by harmonic regression of tasseled cap transformation time series data were shown to be correlated with land cover, including tree cover. Regression models using estimated Fourier coefficients as predictor variables showed a two- to threefold increase in explained variance for a small set of continuous response variables, relative to comparable models using monthly image composites. Similarly, the overall accuracies of classification models using the estimated Fourier coefficients were approximately 10–20 percentage points higher than the models using the image composites, with corresponding individual class accuracies between six and 45 percentage points higher.",
    keywords=["harmonic regression", "landsat", "forest inventory", "forest health","time series"],
    object_class="Link",
    item={
        'href': "https://www.fs.usda.gov/research/treesearch/55701"
    }
)
paper.save()

model_node.connect_with(predicate='links', object=paper)


modelset:remote-sensing:earth-observation:multispectral:harmonic-regression-model --links--> link:*:*:*:har-reg-usfs-paper

In [70]:
p = geodesic.get_objects(search="har-reg-usfs-paper")
p

[link:*:*:*:har-reg-usfs-paper]

In [73]:
p[0].description

'Imagery from the Landsat Program has been used frequently as a source of auxiliary data for modeling land cover, as well as a variety of attributes associated with tree cover. With ready access to all scenes in the archive since 2008 due to the USGS Landsat Data Policy, new approaches to deriving such auxiliary data from dense Landsat time series are required. Several methods have previously been developed for use with finer temporal resolution imagery (e.g. AVHRR and MODIS), including image compositing and harmonic regression using Fourier series. The manuscript presents a study, using Minnesota, USA during the years 2009–2013 as the study area and timeframe. The study examined the relative predictive power of land cover models, in particular those related to tree cover, using predictor variables based solely on composite imagery versus those using estimated harmonic regression coefficients. The study used two common non-parametric modeling approaches (i.e. k-nearest neighbors and ra